# Agents for Code Generation
Autonomous Code Generation with Multi-Agent System

Developed a multi-agent system using Autogen to autonomously generate, execute, and debug Python scripts.

Implemented agents for code generation, execution, error analysis, and user simulation to iteratively improve code quality.

Built a workflow to identify pure Nash equilibria in 2x2 games using best-response logic, leveraging LocalCommandLineCodeExecutor for safe code execution.

Demonstrated the capability of collaborative AI agents to autonomously handle software development tasks with minimal human intervention.

# Setting Up the Development Environment

In [8]:
import tempfile
import autogen
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor
import openai
from openai import OpenAI

# LLM set up

In [1]:
config_list = [
    {
        "model": "gpt-3.5-turbo",
        "api_key": OPENAI_API_KEY,
    }
]

# Python Environment

In [10]:
# Temporary Directory
temp_dir = tempfile.TemporaryDirectory()

# Code Executor
executor = LocalCommandLineCodeExecutor(timeout=10, work_dir=temp_dir.name)

# Agents are all we need

In [11]:
code_executor_agent = ConversableAgent(
    name="CodeExecutorAgent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="ALWAYS"
)
error_analyzer_agent = ConversableAgent(
    name="ErrorAnalyzerAgent",
    llm_config={"config_list": config_list},
    system_message="You analyze errors in Python code and suggest fixes. Provide clear explanations and code improvements. Ensure the code handles edge cases and common errors such as invalid inputs and arithmetic errors."
)
user_proxy = ConversableAgent(
    name="User",
    code_execution_config=False,
    human_input_mode="ALWAYS"
)

# code generator agent

In [12]:
code_generator_agent = ConversableAgent(
    name="CodeGeneratorAgent",
    llm_config={"config_list": config_list},
    system_message="""You are an expert Python programmer with knowledge of game theory. Your task is to develop a Python script that finds pure Nash equilibria in 2x2 games using the best response approach. The program should have a text-based interface and include the following features:

    1. A function to represent a 2x2 game using nested lists.
    2. A function to calculate best responses for each player.
    3. A function to find pure Nash equilibria based on best responses.
    4. A text-based interface for users to input game payoffs and view results.
    5. Clear display of the game, best responses, and Nash equilibria.
    6. Proper error handling and input validation.

    Important Notes:
    - Use nested lists to represent the game: [[[a,b],[c,d]], [[e,f],[g,h]]] where the first sublist is Player 1's payoffs and the second is Player 2's.
    - Implement the best response calculation for each player's strategies, considering cases where strategies might be equally good.
    - Add detailed comments to explain key steps, especially the best response and Nash equilibrium logic.
    - Ensure the text interface is user-friendly and provides clear instructions.
    - Include an option for users to analyze multiple games without restarting the program.

    Best Response Calculation:
    - For each player and each strategy of the other player, determine which strategy yields the highest payoff.
    - If both strategies yield equal payoffs, both should be considered as best responses.
    - Return best responses as a list of lists: [[p1_br1, p1_br2], [p2_br1, p2_br2]], where each br can be 1, 2, or 'Both'.

    Nash Equilibrium Finding Algorithm:
    - A strategy profile is a Nash equilibrium if no player can unilaterally deviate and improve their payoff.
    - Implement the following steps:
      1. Iterate through all four possible strategy combinations: (1,1), (1,2), (2,1), (2,2).
      2. For each combination (i,j):
         - Check if strategy i is a best response for Player 1 to Player 2's strategy j
         - AND if strategy j is a best response for Player 2 to Player 1's strategy i
      3. If both conditions are true, add (i,j) to the nash_equilibria list.
    - Handle cases where multiple strategies might be equally good (i.e., when best_responses contains 'Both').
    - Ensure the function works correctly for games with 0, 1, or 2 Nash equilibria.

    Text Interface:
    - Prompt the user to enter payoffs for both players.
    - Display the game in a readable format.
    - Show the best responses for each player.
    - Clearly indicate the pure Nash equilibria found.
    - Allow the user to input multiple games or quit the program.

    Example Output Format (using Prisoner's Dilemma):
    ```
    Enter payoffs for Player 1:
    Strategy 1,1: -1
    Strategy 1,2: -3
    Strategy 2,1: 0
    Strategy 2,2: -2

    Enter payoffs for Player 2:
    Strategy 1,1: -1
    Strategy 1,2: 0
    Strategy 2,1: -3
    Strategy 2,2: -2

    Game:
    Player 1's payoffs:    Player 2's payoffs:
    [-1, -3]               [-1, 0]
    [0, -2]                [-3, -2]

    Best Responses:
    Player 1: [2, 2]
    Player 2: [2, 2]

    Pure Nash Equilibria: [(2,2)]
    ```

    Ensure your implementation is correct, user-friendly, and provides clear output for analysis of 2x2 games using the best response approach. Pay special attention to the correct implementation of the Nash equilibrium finding algorithm and best response calculation.
    """
)

# Group Chat Initialization

In [15]:
# Group Chat Initialization
group_chat = GroupChat(agents=[code_generator_agent, code_executor_agent, error_analyzer_agent, user_proxy], messages=[])
manager = GroupChatManager(groupchat=group_chat, llm_config={"config_list": config_list})

# Error Handling

In [ ]:
# Error Handling
def handle_execution_error(message, e):
    error_message = f"Code execution failed:\n```\n{message.content}\n```\nError:\n```\n{e}\n```"
    error_analyzer_agent.send(error_message, group_chat)

# Clear existing replies and register error handler
code_executor_agent._reply_func_list.clear()
code_executor_agent.register_reply([autogen.AssistantAgent, LocalCommandLineCodeExecutor], handle_execution_error)

# Enhanced General Error Analysis
def enhanced_error_analysis(message, error):
    error_type = type(error).__name__
    suggestions = {
        'ValueError': "Check for invalid inputs and ensure all values are within expected ranges.",
        'ZeroDivisionError': "Ensure there are no divisions by zero. Add checks to prevent zero as a denominator.",
        'TypeError': "Verify that all operations are being performed on compatible data types.",
        'IndexError': "Ensure that list or array indices are within the valid range.",
        'KeyError': "Check that dictionary keys exist before accessing them.",
        'AttributeError': "Verify that the objects have the attributes or methods being accessed.",
        'ImportError': "Ensure all required modules are installed and correctly imported."
    }
    suggestion = suggestions.get(error_type, "Review the code for potential issues.")

    error_message = f"Code execution failed:\n```\n{message.content}\n```\nError:\n```\n{error}\n```\nSuggested fix:\n```\n{suggestion}\n```"
    error_analyzer_agent.send(error_message, group_chat)

# Error analysis registering
code_executor_agent.register_reply([autogen.AssistantAgent, LocalCommandLineCodeExecutor], enhanced_error_analysis)

# Finally user proxy agent is responsible for starting conversation.

In [ ]:
# Start Conversation
user_proxy.initiate_chat(
    manager, message="Generate Python code to find pure Nash equilibria in 2x2 games using the best response approach. Implement a text-based interface for user input and result display. Include functions for game representation, best response calculation, and equilibrium finding based on best responses."
)